# 0. Intro

### On my mind (in case I wake up with dementia :D)
I want to build an RSA. For this I have several parameters to consider, so I have to do this per subject, contrast (of all tasks), per hemisphere.

If there is more than one session of the same task_contrast (space&subject&task&contrast&hemisphere), I guess I should average them?

I have the Glasser atlas in fsaverage space (in which I want to work) and the parcels that I want to use (only Frontoparietal). I need to map whe voxels from the atlas to all parcels, and then only keep the FPN parcels.

With the info on which vertices to inlcude (FPN) I want to shorten the contrast maps from whole brain activation to activation of FPN vertices. Then I will seperate those to get beta maps per parcel, per contrast.

Question: When looking at parcel X and contrast Y, how so I get from a matrix of *vertices x task* to *task x task* correlation matrix

At this point I am still witin a single subject.

In [2]:
import os
import nibabel as nib
from nibabel.freesurfer.io import read_annot
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr

base_dir = '/home/hmueller2/Downloads/contrast_maps/resulting_smooth_maps_surface/' # find the contrast maps here
output_dir = '/home/hmueller2/ibc_code/ibc_output_RSA'
code_dir = '/home/hmueller2/ibc_code/ibc_latent'

# 1. Load atlas and extract FPN parcels

## 1.1 Glasser: vertex-to-parcel mappings
1. **Vertex Labels**: Each vertex on the cortical surface mesh is assigned a label indicating the region or parcel it belongs to (No.: 2 x 163842).
2. **Color Table**: A mapping of labels to colors, which helps in visualizing the different regions on the cortical surface.
3. **Region Names**: Names of the regions or parcels corresponding to the labels (No.: 2 x 181).


In [47]:
# Glasser atlas (already in fsaverage space)
'''
Naming convention in Glasser:
- R/L: Indicates the hemisphere. R stands for the right hemisphere, and L stands for the left hemisphere.
- Parcel Name: The name following the hemisphere indicator (e.g., 6mp, AIP, V4) represents a specific cortical area. These names are often based on anatomical or functional characteristics.
- ROI: Stands for "Region of Interest," indicating that the parcel is a specific region within the brain.
'''

# Load the annotation files
lh_annot_file = '/home/hmueller2/Downloads/Atlas/glasser_fsaverage/3498446/lh.HCP-MMP1.annot'
rh_annot_file = '/home/hmueller2/Downloads/Atlas/glasser_fsaverage/3498446/rh.HCP-MMP1.annot'

# Read the annotation files
labels_lh, ctab_lh, names_lh = read_annot(lh_annot_file)
labels_rh, ctab_rh, names_rh = read_annot(rh_annot_file)

# Overview on annot (for left hemisphere, but euqally applies to right hemisphere)
# Labels: Length of labels: 163842 (= amount of vertices in the left hemisphere in fsaverage)
print('Length of left hemisphere labels:', len(labels_lh))
print('First 15 labels of left hemisphere:', labels_lh[:15])

# Parcels: Number of unique labels: 181 (= amount of parcels in the left hemisphere)
unique_labels_lh = np.unique(labels_lh)
print('Amount of unique labels in left hemisphere:', len(unique_labels_lh))

Length of left hemisphere labels: 163842
First 15 labels of left hemisphere: [ 54  49  41  86  99 149   6 121   0 111 129 135   8  83  97]
Amount of unique labels in left hemisphere: 181


In [ ]:
# Create vertex-to-parcel mappings in Glasser atlas
vertices_lh = np.arange(len(labels_lh))
vertices_rh = np.arange(len(labels_rh))
lh_parcel_mapping = {vertex: names_lh[label] for vertex, label in zip(vertices_lh, labels_lh)}
rh_parcel_mapping = {vertex: names_rh[label] for vertex, label in zip(vertices_rh, labels_rh)}

# Print the first mappings
print("Left Hemisphere Parcel Mapping (first 5 entries):")
print({k: lh_parcel_mapping[k] for k in list(lh_parcel_mapping)[:5]})

print("Right Hemisphere Parcel Mapping (first 5 entries):")
print({k: rh_parcel_mapping[k] for k in list(rh_parcel_mapping)[:5]})

Left Hemisphere Parcel Mapping (first 5 entries):
{np.int64(0): np.bytes_(b'L_6d_ROI'), np.int64(1): np.bytes_(b'L_VIP_ROI'), np.int64(2): np.bytes_(b'L_24dv_ROI'), np.int64(3): np.bytes_(b'L_9-46d_ROI'), np.int64(4): np.bytes_(b'L_43_ROI')}
Right Hemisphere Parcel Mapping (first 5 entries):
{np.int64(0): np.bytes_(b'R_6mp_ROI'), np.int64(1): np.bytes_(b'R_AIP_ROI'), np.int64(2): np.bytes_(b'R_1_ROI'), np.int64(3): np.bytes_(b'R_46_ROI'), np.int64(4): np.bytes_(b'R_p32pr_ROI')}


## 1.2 Network Partition: FPN parcels-vertices

In [48]:
# Outdated: Load Excel file containing labels of parcels
# Outdated: network_partition_path = '/home/hmueller2/Downloads/Cole_FPN_Parcellation/CAB-NP_v1.1_Labels-ReorderedbyNetworks.xlsx'

# Load the text file containing labels of parcels
network_partition_path = '/home/hmueller2/Downloads/Cole_FPN_Parcellation/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_LabelKey.txt'
network_partition = pd.read_csv(network_partition_path, sep='\t')

# Give out overall amount of parcels
print(f"Total number of parcels (= unique labels): {len(network_partition)}")
print(f"Number of unique networks (l&r): {network_partition['NETWORK'].nunique()}")
print(f"Number of unique glasser labels (l&r): {network_partition['GLASSERLABELNAME'].nunique()}")
network_partition.head()

Total number of parcels (= unique labels): 718
Number of unique networks (l&r): 12
Number of unique glasser labels (l&r): 360


,INDEX,KEYVALUE,LABEL,RED,GREEN,BLUE,ALPHA,HEMISPHERE,NETWORK,NETWORKKEY,NETWORKSORTEDORDER,GLASSERLABELNAME
0,1,1,Visual1-04_L-Ctx,0,0,255,255,L,Visual1,1,1,L_V1_ROI
1,2,2,Visual2-28_L-Ctx,100,0,255,255,L,Visual2,2,70,L_MST_ROI
2,3,3,Visual2-29_L-Ctx,100,0,255,255,L,Visual2,2,71,L_V6_ROI
3,4,4,Visual2-30_L-Ctx,100,0,255,255,L,Visual2,2,72,L_V2_ROI
4,5,5,Visual2-31_L-Ctx,100,0,255,255,L,Visual2,2,73,L_V3_ROI


In [66]:
# Extract parcels of FPN
fpn_parcels = network_partition[network_partition['NETWORK'] == 'Frontoparietal']

# Give out amount of FPN parcels (98, where 46 are in the left hemisphere and 50 in the right hemisphere)
print(f"Number of labels within FPN: {len(fpn_parcels)}")
print(f"Number of unique glasser labels from FPN: {fpn_parcels['GLASSERLABELNAME'].nunique()}")
fpn_parcels.head()

Number of labels within FPN: 98
Number of unique glasser labels from FPN: 50


,INDEX,KEYVALUE,LABEL,RED,GREEN,BLUE,ALPHA,HEMISPHERE,NETWORK,NETWORKKEY,NETWORKSORTEDORDER,GLASSERLABELNAME
13,14,14,Frontoparietal-29_L-Ctx,255,255,0,255,L,Frontoparietal,7,399,L_RSC_ROI
14,15,15,Frontoparietal-30_L-Ctx,255,255,0,255,L,Frontoparietal,7,400,L_POS2_ROI
28,29,29,Frontoparietal-31_L-Ctx,255,255,0,255,L,Frontoparietal,7,401,L_7Pm_ROI
62,63,63,Frontoparietal-32_L-Ctx,255,255,0,255,L,Frontoparietal,7,402,L_8BM_ROI
72,73,73,Frontoparietal-33_L-Ctx,255,255,0,255,L,Frontoparietal,7,403,L_8C_ROI


In [67]:
# Inspect parcels from left and right hemisphere of FPN separately
fpn_parcels_lh = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'L']
fpn_parcels_rh = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'R']
fpn_parcels_both = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'LR']
print(f"Number of Glasser parcels from left FPN: {fpn_parcels_lh['GLASSERLABELNAME'].nunique()}")
print(f"Number of Glasser parcels from right FPN: {fpn_parcels_rh['GLASSERLABELNAME'].nunique()}")
print(f"Number of Glasser parcels from FPN inbetween hemisphere: {fpn_parcels_both['GLASSERLABELNAME'].nunique()}")
fpn_parcels_rh.head()

Number of Glasser parcels from left FPN: 22
Number of Glasser parcels from right FPN: 28
Number of Glasser parcels from FPN inbetween hemisphere: 0


,INDEX,KEYVALUE,LABEL,RED,GREEN,BLUE,ALPHA,HEMISPHERE,NETWORK,NETWORKKEY,NETWORKSORTEDORDER,GLASSERLABELNAME
193,194,194,Frontoparietal-01_R-Ctx,255,255,0,255,R,Frontoparietal,7,421,R_RSC_ROI
194,195,195,Frontoparietal-02_R-Ctx,255,255,0,255,R,Frontoparietal,7,422,R_POS2_ROI
208,209,209,Frontoparietal-03_R-Ctx,255,255,0,255,R,Frontoparietal,7,423,R_7Pm_ROI
237,238,238,Frontoparietal-04_R-Ctx,255,255,0,255,R,Frontoparietal,7,424,R_33pr_ROI
241,242,242,Frontoparietal-05_R-Ctx,255,255,0,255,R,Frontoparietal,7,425,R_d32_ROI


In [51]:
import pandas as pd

# Use atlas annotation (lh_parcel_mapping & rh_parcel_mapping) and only keep parcels of FPN
# Use GLASSERLABELNAME in fpn_parcels to match with lh_parcel_mapping & rh_parcel_mapping

# Left hemisphere: all vertices assigned to parcels of FPN
fpn_parcels_lh_names = list(fpn_parcels_lh['GLASSERLABELNAME'].dropna())
fpn_parcels_lh_mapping = {vertex: lh_parcel_mapping[vertex] for vertex in vertices_lh if lh_parcel_mapping[vertex].decode('utf-8') in fpn_parcels_lh_names}

# Right hemisphere: all vertices assigned to parcels of FPN
fpn_parcels_rh_names = list(fpn_parcels_rh['GLASSERLABELNAME'].dropna())
fpn_parcels_rh_mapping = {vertex: rh_parcel_mapping[vertex] for vertex in vertices_rh if rh_parcel_mapping[vertex].decode('utf-8') in fpn_parcels_rh_names}

# Check if the number of parcels in the mapping is correct
print(f"Number of vertices within parcels of FPN in left hemisphere mapping: {len(fpn_parcels_lh_mapping)}")
print(f"Number of vertices within parcels of FPN in right hemisphere mapping: {len(fpn_parcels_rh_mapping)}")

# Visualization of vertex - parcel mapping
# Convert the mappings to DataFrames for better visualization
lh_mapping_df = pd.DataFrame(list(fpn_parcels_lh_mapping.items()), columns=['Vertex', 'Parcel'])
rh_mapping_df = pd.DataFrame(list(fpn_parcels_rh_mapping.items()), columns=['Vertex', 'Parcel'])

# Display a few entries of the mappings (here left hemisphere)
print("Left Hemisphere Parcel Mapping:")
lh_mapping_df

Number of vertices within parcels of FPN in left hemisphere mapping: 18361
Number of vertices within parcels of FPN in right hemisphere mapping: 22966
Left Hemisphere Parcel Mapping:


,Vertex,Parcel
0,5,b'L_PFm_ROI'
1,9,b'L_AVI_ROI'
2,13,b'L_p9-46v_ROI'
3,14,b'L_i6-8_ROI'
4,21,b'L_8BM_ROI'
...,...,...
18356,162034,b'L_TE1p_ROI'
18357,162035,b'L_TE1p_ROI'
18358,162036,b'L_TE1p_ROI'
18359,162037,b'L_TE1p_ROI'


# 2. Load contrast data

In [43]:
# Parameters
space = 'fsaverage7' # 'fsaverage5' or 'fsaverage7'

subject = '01'
session = '00'
task = 'ArchiStandard'
contrast = 'audio_computation'
hemisphere = 'lh' # 'lh' or 'rh'

In [ ]:
# For Pipeline
'''
def load_surface_maps(file_paths):
    data = []
    for file_path in file_paths:
        img = nib.load(file_path)
        data.append(img.get_fdata())
    return np.array(data)

file_paths = [f"{base_dir}/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_dir-ffx_space-{space}_hemi-{hemisphere}_ZMap-{contrast}.gii"]
data = load_surface_maps(file_paths)
'''

SyntaxError: incomplete input (1228653324.py, line 2)

In [56]:
# Load gifti and output shape -> amount of vertices / data points (163842)

file_path = [f"{base_dir}/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_dir-ffx_space-{space}_hemi-{hemisphere}_ZMap-{contrast}.gii"]
img = nib.load(file_path[0])
data = np.array([darray.data for darray in img.darrays])
print(f"Shape of data: {data.shape}")
data

Shape of data: (1, 163842)


array([[-2.86166986,  0.27651445, -3.55043359, ..., -1.87416169,
        -1.80440628, -1.53897857]], shape=(1, 163842))

In [ ]:
'''
# In case of same subject, same contrast but different session, average the data
# Define the file paths for all sessions
file_paths = [
    f"{base_dir}/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_dir-ffx_space-{space}_hemi-{hemisphere}_ZMap-{contrast}.gii",
    # Add more sessions if needed
]

# Load the data for all sessions
data_sessions = [nib.load(fp).darrays[0].data for fp in file_paths]

# Average the data across sessions
average_data = np.mean(data_sessions, axis=0)

# Print the shape of the averaged data
print(f"Shape of averaged data: {average_data.shape}")
average_data
'''

In [72]:
# CHECK: ONLY 22 PARCELS BUT 46 EXPECTED

# Extract FPN areas from the contrast maps using the mappings

# Initialize a dictionary to store the activations for each parcel
fpn_parcel_activations = {}

# Determine the parcel mapping based on the hemisphere
if hemisphere == 'lh':
    parcel_mapping = fpn_parcels_lh_mapping
else:
    parcel_mapping = fpn_parcels_rh_mapping

# Iterate over each parcel in the mapping
for vertex, parcel in parcel_mapping.items():
    parcel_name = parcel.decode('utf-8')
    if parcel_name not in fpn_parcel_activations:
        fpn_parcel_activations[parcel_name] = []
    fpn_parcel_activations[parcel_name].append(data[:, vertex])

# Convert lists to numpy arrays
for parcel_name in fpn_parcel_activations:
    fpn_parcel_activations[parcel_name] = np.array(fpn_parcel_activations[parcel_name])

# Print the shape of the extracted data for each parcel
for parcel_name, activations in fpn_parcel_activations.items():
    print(f"Parcel: {parcel_name}, Shape of activations: {activations.shape}")

# Check the number of unique parcels
print(f"Number of unique parcels: {len(fpn_parcel_activations)}")

# Print the actual array of activations for the parcel 'L_PFm_ROI'
parcel_to_print = 'L_AVI_ROI'
if parcel_to_print in fpn_parcel_activations:
    print(f"Activations for parcel '{parcel_to_print}':")
    print(fpn_parcel_activations[parcel_to_print])
else:
    print(f"Parcel '{parcel_to_print}' not found in the activations dictionary.")

Parcel: L_PFm_ROI, Shape of activations: (2451, 1)
Parcel: L_AVI_ROI, Shape of activations: (716, 1)
Parcel: L_p9-46v_ROI, Shape of activations: (794, 1)
Parcel: L_i6-8_ROI, Shape of activations: (549, 1)
Parcel: L_8BM_ROI, Shape of activations: (781, 1)
Parcel: L_a47r_ROI, Shape of activations: (915, 1)
Parcel: L_POS2_ROI, Shape of activations: (1209, 1)
Parcel: L_RSC_ROI, Shape of activations: (1092, 1)
Parcel: L_p10p_ROI, Shape of activations: (549, 1)
Parcel: L_IFSa_ROI, Shape of activations: (666, 1)
Parcel: L_IP2_ROI, Shape of activations: (1000, 1)
Parcel: L_7Pm_ROI, Shape of activations: (490, 1)
Parcel: L_TE1p_ROI, Shape of activations: (1173, 1)
Parcel: L_IFJp_ROI, Shape of activations: (502, 1)
Parcel: L_8C_ROI, Shape of activations: (1042, 1)
Parcel: L_s6-8_ROI, Shape of activations: (307, 1)
Parcel: L_13l_ROI, Shape of activations: (791, 1)
Parcel: L_11l_ROI, Shape of activations: (713, 1)
Parcel: L_IP1_ROI, Shape of activations: (819, 1)
Parcel: L_a9-46v_ROI, Shape of act

In [ ]:
### Now turn into pipeline so that you can loop over all subjects, sessions, tasks, contrasts and hemispheres

# Needed to come up with RSMs (contrast x contrast)